## Data Extraction Using Mixtral:7x8b

**This is a data extraction project in which we extract product attributes of individual product categories one by one.**
**Problem**: 
- There is a large set of HTML text in an excel sheet that has all the product attributes of our products. <br>
- The product attributes are not individually listed anywhere. <br>
- We need a dataset of product attributes in which various properties of each product are listed separately, in order to build a PIM system, and to use specific attributes to SEO-optimize our webshop texts. <br>
- The Challenge is that the text is not regular, different attributes come in a variety of formats from one product to the next. <br>
- Another challenge is that we do not have a comprehensive list of product properties and that also needs to be created along the way. <br>

**Solution**: 
1. The first solution was Text mining using Regular expressions. It was implemented on one product group by reading and analyzing product descriptions of many products and finding attributes and then generating the regex patterns to extract them. <br>
Successful, but took a lot of time and energy. 
2. The second solution was to use SpaCy and NLP methods to extract adjective and prepositional groups such as "mit Griff" or "mit Deckel" to then use them for attribute generation. <br>
This was a faster method than raw text mining, but the problem was a large number of false positives (adjectives that were not product attributes) and false negatives <br>
(items that were not adjectives or prepositional groups but were attributes of the product nevertheless). 
3. This lead to trying out LLMs. The first attempt was with the transformers library. However, running into Langchain and Ollama, I found them to be faster solutions. <br>
I used Ollama because it supported the Mixtral:7x8b model which is both compatible with structured outputs and also supports German language. <br>
The result is the following code that functions much better in extracting all the relevant data required for our products.

- Note: The prompt was an important part of the modeling which resulted in correct and coherent results that could be further processed.

In [12]:
## Importing libraries
import pandas as pd
import re
import json
from values import *
import ollama
import warnings
import os
from mistralai import Mistral

warnings.filterwarnings('ignore')



In [13]:
## Loading Productgroup Number and its name
val = Values()
wg_list = val.wr_gr
wg_name = val.wr_name
client = val.client
model = val.model

In [ ]:
## loading the csv file that has the product information for each warengruppe (previously generated using wr_folder_building.ipynb)
file = pd.read_csv(f'{val.parent_dir}{wg_list}/{wg_list}.csv',delimiter=';',encoding='utf-8')
file = pd.read_csv(f'tischdecke-online.csv',delimiter=';',encoding='utf-8')
mined_text = file.copy()
mined_text

In [5]:
## filtering only the necessary columns
mined_text = mined_text[['NUMMER','NAME','NET_BESCHREIBUNG']]
# mined_text = mined_text[:10]

In [6]:
allowed_keys = [
    "NUMMER", # Produkt ID
    "NAME", # 1-Stufen Sicherheitsleiter
    "MATERIAL", # Aluminium, Kunststoff
    "PRODUKTART", # Wasserkocher, Mobiler Grillwagen
    "FARBE-TEXT", # Rot
    "GROESSE-TEXT", # klein, groß
    "HERGESTELLT", # Deutschland
    "SET", # 3-tlg.
    "STUECK", # 1 Stück
    "DESIGN", # Extrabreite Stufen mit Antirutschprofil
    "MOTIV", # Blumenmotiv
    "MUSTER", # großer Pfingstrosen-Print
    "FORM", # Rund, Quadratisch
    "GEWEBEART", #Baumwolle
    "BREITE", # 31,5 cm (obere Standfläche)
    "LAENGE", # 14,5 cm (Trittstufen)
    "DURCHMESSER", # Ø 20cm
    "ANDERE-ABMESSUNGEN", # [ 10 x 15 x 5 cm (B/T/H) ]
    "PFLEGEHINWEIS", # Washbarkeit und trocknerkeit
    "SCHUTZ", # Teflon (TM)-Ausrüstung , Fleckenschutz
    "BRAND", # Hagen Grote
    "QUALITAET", # Hochfeste Aluminiumstruktur
    "ZERTIFIKATE", # OEKO-TEX® STANDARD 100, BIO
    "ANDERE-EIGENSCHAFTEN" # ["Standsicher", "Superleicht", "Ultraflach", "Fester Stand", "Oberer Haltebügel", "Geprüfte Sicherheit"]
]

In [25]:
## function to clean responses from the LLM
def clean_response(response_text):
    response_text = response_text.replace(r"\\u00fc","ü")
    try:
        response_dict = json.loads(response_text)
        cleaned_dict = {key: value.strip() if isinstance(value, str) else value for key, value in response_dict.items()}
        return cleaned_dict

    except json.JSONDecodeError :

        print(f"Error: The response is not a valid JSON object.")
        print(response_text)

        return None

In [29]:
def clean_response(response_text):
    response_text = response_text.replace(r"\\u00fc", "ü")
    try:
        response_dict = json.loads(response_text)

        # Check if response_dict is a dictionary
        if isinstance(response_dict, dict):
            cleaned_dict = {key: value.strip() if isinstance(value, str) else value for key, value in response_dict.items()}
            return cleaned_dict

        # Check if response_dict is a list
        elif isinstance(response_dict, list):
            # Clean list elements if they are strings
            cleaned_list = [item.strip() if isinstance(item, str) else item for item in response_dict]
            return cleaned_list

        # If response_dict is neither dict nor list, return it as is
        else:
            return response_dict

    except json.JSONDecodeError:
        print(f"Error: The response is not a valid JSON object.")
        print(response_text)
        return None

# Original model - not modified

In [8]:
# ### Loading the prompt and generating responses based on previous variables as well as extact instrctions
# uncleaned_data = []

# for id,name, net_beschreibung,kat_beshreibung in zip(mined_text['NUMMER'],mined_text['NAME'],mined_text['NET_BESCHREIBUNG'],mined_text['KAT_BESCHREIBUNG']):
#     print(id,name)

#     prompt = f"""[INST]
#                 1. Extrahieren Sie die Produktattribute aus der folgenden Net_Beschreibung und geben Sie sie als gültiges JSON-Objekt in deutscher Sprache aus.
#                 2. Verwenden Sie NUR die allowed_keys = {allowed_keys} Schlüssel und ändern Sie diese nicht.
#                 3. Katalogue_beschreibung wird NUR berücksichtigt, wenn ein für das Modell erforderlicher Wert fehlt. Andernfalls überspringe sie.
#                 4. Nutzen Sie ALLE erlaubten Schlüssel im JSON-Objekt und setzen Sie nicht verfügbare Werte als "N/A".
#                 5. Beschränken Sie die Werte auf maximal 50 Zeichen.
#                 6. Verwenden Sie NUR die im Text vorhandenen Informationen, ohne Schlüsse oder Vermutungen.
#                 7. Das JSON-Objekt sollte KEINE ASCII-Kodierung enthalten.
#                 8. Überprüfen Sie doppelt, ob die Schlüssel alle genau gleich als "allowed_keys" sind.
#                 9. Achten Sie darauf, dass die Reihenfolge der Dimensionen B/T/H ist (Breite, Tiefe, Höhe).
#                 10. Werte sollten eindeutige Wörter sein und keine "Ja", "Nein", "True" oder "False". Beispiel: DECKEL = "Mit Deckel".
#                 11. Die Ausgabe sollte NUR das JSON-Objekt sein, ohne zusätzlichen Text außerhalb von geschweiften Klammern.
#                 12. Verwenden Sie ausschließlich Informationen aus dem aktuellen Text, ohne Vorwissen oder Ableitungen.
#                 [/INST]
                    
#                     NUMMER: {id}
#                     NAME: {name}
#                     Net_beschreibung: {net_beschreibung}
#                     Katalogue_beschreibung:{kat_beshreibung}
#         """

# # Generate response using Mixtral 7x8b
#     response = ollama.generate(model='mixtral:latest', prompt=prompt)

# # Clean up the response
#     uncleaned_data.append(response['response'])



# Modified model, with Mistral API

In [ ]:
### Loading the prompt and generating responses based on previous variables as well as extact instrctions
uncleaned_data = []

for id,name, net_beschreibung in zip(mined_text['NUMMER'],mined_text['NAME'],mined_text['NET_BESCHREIBUNG']):
    print(id,name)

    prompt = f"""[INST]
                1. Extrahieren Sie die Produktattribute aus der folgenden Net_Beschreibung und geben Sie sie als gültiges JSON-Objekt in deutscher Sprache aus.
                2. Verwenden Sie NUR die allowed_keys = {allowed_keys} Schlüssel und ändern Sie diese nicht.
                3. Katalogue_beschreibung wird NUR berücksichtigt, wenn ein für das Modell erforderlicher Wert fehlt. Andernfalls überspringe sie.
                4. Nutzen Sie ALLE erlaubten Schlüssel im JSON-Objekt und setzen Sie nicht verfügbare Werte als "N/A".
                5. Beschränken Sie die Werte auf maximal 50 Zeichen.
                6. Verwenden Sie NUR die im Text vorhandenen Informationen, ohne Schlüsse oder Vermutungen.
                7. Das JSON-Objekt sollte KEINE ASCII-Kodierung enthalten.
                8. Überprüfen Sie doppelt, ob die Schlüssel alle genau gleich als "allowed_keys" sind.
                9. Achten Sie darauf, dass die Reihenfolge der Dimensionen B/T/H ist (Breite, Tiefe, Höhe).
                10. Werte sollten eindeutige Wörter sein und keine "Ja", "Nein", "True" oder "False". Beispiel: DECKEL = "Mit Deckel".
                11. Die Ausgabe sollte NUR das JSON-Objekt sein, ohne zusätzlichen Text außerhalb von geschweiften Klammern.
                12. Verwenden Sie ausschließlich Informationen aus dem aktuellen Text, ohne Vorwissen oder Ableitungen.
                [/INST]
                    
                    NUMMER: {id}
                    NAME: {name}
                    Net_beschreibung: {net_beschreibung}
                  
        """

    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    chat_response = client.chat.complete(
        model = model,
        messages = messages,
        response_format = {
            "type": "json_object",
        }
    )

    print(chat_response.choices[0].message.content)
# # Generate response using Mixtral 7x8b
#     response = ollama.generate(model='mixtral:latest', prompt=prompt)

# # Clean up the response
    uncleaned_data.append(chat_response.choices[0].message.content)









In [33]:
## cleaning one step further and appending responses to a new list
new_data = []
for id,item in enumerate(uncleaned_data):
    item = item.replace('```',"")
    item = item.replace('json',"")
    # print(clean_response(item))
    new_data.append(clean_response(item))
    

In [35]:
## saving clean json-objects to a json file
with open(f'{val.parent_dir}{val.wr_gr}/{val.wr_name}_{val.wr_gr}.json', 'w', encoding="utf-8") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)    